**import libraries**

In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [ ]:
# for Emoticons score features
!pip install afinn

In [ ]:
# for VADER Sentiment feature
!pip install vaderSentiment

In [ ]:
# for Aggregate emotion score
!pip install NRCLex

In [ ]:
import pandas as pd
import re
from collections import defaultdict, Counter
import csv
import numpy as np

from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords 
from nltk.util import everygrams
import itertools

#for sentiment analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#for n-grams
from sklearn.feature_extraction.text import CountVectorizer

#ML models
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor

#evaluate
from scipy.stats import pearsonr
from scipy.stats import spearmanr

######**Load dataset**

In [ ]:
anger_train = pd.read_csv('/content/drive/My Drive/NLP_AssgnData/anger_train.csv')
anger_train

,ID,Tweet_text,Emotion,Intensity
0,10000,How the fu*k! Who the heck! moved my fridge!.....,anger,0.938
1,10001,So my Indian Uber driver just called someone t...,anger,0.896
2,10002,@DPD_UK I asked for my parcel to be delivered ...,anger,0.896
3,10003,so ef whichever butt wipe pulled the fire alar...,anger,0.896
4,10004,Don't join @BTCare they put the phone down on ...,anger,0.896
...,...,...,...,...
852,10852,rose incense are the best thing I've ever bought,anger,0.125
853,10853,@jaaames1993 Literally burst out laughing.,anger,0.067
854,10854,Follow up. Follow through. Be . #success,anger,0.125
855,10855,Wrinkles should merely hide where frown have b...,anger,0.125


In [ ]:
joy_train = pd.read_csv('/content/drive/My Drive/NLP_AssgnData/joy_train.csv')
joy_train

,ID,Tweet_text,Emotion,Intensity
0,30000,Just got back from seeing @GaryDelaney in Burs...,joy,0.980
1,30001,Oh dear an evening of absolute hilarity I don'...,joy,0.958
2,30002,Been waiting all week for this game ❤️❤️❤️ #ch...,joy,0.940
3,30003,"@gardiner_love : Thank you so much, Gloria! Yo...",joy,0.938
4,30004,I feel so blessed to work with the family that...,joy,0.938
...,...,...,...,...
818,30818,It's just the lack of company and liveliness o...,joy,0.058
819,30819,Quinn's short hair makes me sad. #glee,joy,0.040
820,30820,hate overthinking e v e r y t h i n g like i j...,joy,0.040
821,30821,People who cheer for sports teams completely o...,joy,0.020


In [ ]:
# #concatenate the two dataframes
# train_data = pd.concat([anger_train,joy_train])

# #shuffle the rows of the concatenated dataframe
# train_data = train_data.sample(frac=1)
# train_data[:10]

**Test Data**

In [ ]:
anger_test = pd.read_csv('/content/drive/My Drive/NLP_AssgnData/anger_test.csv')
anger_test

,ID,Tweet_text,Emotion,Intensity
0,10941,At the point today where if someone says somet...,anger,0.319
1,10942,@CorningFootball IT'S GAME DAY!!!! T MIN...,anger,0.144
2,10943,This game has pissed me off more than any othe...,anger,0.898
3,10944,@spamvicious I've just found out it's Candice ...,anger,0.271
4,10945,@moocowward @mrsajhargreaves @Melly77 @GaryBar...,anger,0.646
...,...,...,...,...
755,11696,@ggreenwald What if the supposed animosity is ...,anger,0.646
756,11697,Will BYU's offense score 24+ vs WVU?,anger,0.125
757,11698,Id love 2 c Gyimah in action but his coach is ...,anger,0.542
758,11699,Forgiving means operating with God's spirit &a...,anger,0.250


In [ ]:
joy_test = pd.read_csv('/content/drive/My Drive/NLP_AssgnData/joy_test.csv')
joy_test

,ID,Tweet_text,Emotion,Intensity
0,30902,You must be knowing #blithe means (adj.) Happ...,joy,0.583
1,30903,Old saying 'A #smile shared is one gained for ...,joy,0.500
2,30904,Bridget Jones' Baby was bloody hilarious 😅 #Br...,joy,0.860
3,30905,@Elaminova sparkling water makes your life spa...,joy,0.521
4,30906,I'm tired of everybody telling me to chill out...,joy,0.042
...,...,...,...,...
709,31611,With a very tired body and mind and sparkling ...,joy,0.417
710,31612,I refuse to be a chirp chirp girl,joy,0.146
711,31613,It was very hard to stifle my laughter after I...,joy,0.646
712,31614,"While I was walking, a little boy in a red shi...",joy,0.620


In [ ]:
# #concatenate the two dataframes
# test_data = pd.concat([anger_test,joy_test])

# #shuffle the rows of the concatenated dataframe
# test_data = test_data.sample(frac=1)
# test_data

######**Pre-processing**

_remove URLs_

In [ ]:
def remove_url(text):
  text = re.sub(r'(\w+:\/\/\S+)','',text)
  
  return text

_remove punctuations_

In [ ]:
def remove_specialChars(text):
  text = (re.sub(r'[\.•,;:?!#=+$%^&*_~><\-\(\)\{\}\[\]\"\|\/\\]','',text)).replace('RT','')
  text = re.sub('\n','',text)
  # changing to lower case
  text = text.lower()
  return text

_remove mentions_

In [ ]:
def remove_mentions(sent):
  sent = re.sub(r'(@[A-Za-z0-9]+)','',sent)
  return sent

_strip off # symbol_

In [ ]:
def remove_hash(sent):
  regex = '#[A-Za-z0-9]+'             
  match = re.findall(regex,sent)         #find all the occurances of #tags

  for item in match:
    stripped = item.strip('#')
    sent = re.sub(item,stripped,sent)     #replace the hastagged words
  
  return sent

In [ ]:
remove_hash("My name is #samee8989 and #gu32pta")

'My name is samee8989 and gu32pta'

_replace elongated words_

In [ ]:
#Replace elongated words by identifying those repeated characters and then remove them and compare the new word with the english lexicon
def in_dict(word):
    if wordnet.synsets(word):
        #if the word is in the dictionary, we'll return True
        return True

def replace_elongated_word(word):
    regex = r'(\w*)(\w+)\2(\w*)'
    repl = r'\1\2\3'    
    if in_dict(word):
        return word
    new_word = re.sub(regex, repl, word)
    if new_word != word:
        return replace_elongated_word(new_word)
    else:
        return new_word

def detect_elongated_words(row):
    regexrep = r'(\w*)(\w+)(\2)(\w*)'
    words = [''.join(i) for i in re.findall(regexrep, row)]
    for word in words:
        if not in_dict(word):
            row = re.sub(word, replace_elongated_word(word), row)
    return row

In [ ]:
detect_elongated_words('Hellooooo guyyyysss')

'Hello guys'

_Expanding contractions_
*you need to convert to lower case before this

In [ ]:
# https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python

contractions_dict = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"i'd": "i had / i would",
"i'd've": "i would have",
"i'll": "i shall / i will",
"i'll've": "i shall have / i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

def expand_contractions(sent):
  for word in sent.split():
    if(word in contractions_dict.keys()):
      sent = re.sub(word,contractions_dict[word],sent)

  return sent

In [ ]:
expand_contractions("you're done with i'd work today")

'you are done with i had / i would work today'

_handling verb present participle form_

In [ ]:
def handle_verbForms(sent):
  regex = '[A-Za-z]+in'
  match = re.findall(regex,sent)

  for w in match:
    # print(w)
    s = w+'g'
    sent = re.sub(w,s,sent)

  return sent

In [ ]:
handle_verbForms("I am comin tomorrow. I am feelin excited!!")

'I am coming tomorrow. I am feeling excited!!'

_remove stopwords_

In [ ]:
def removeStop(tokenst):
  stop = set(stopwords.words('english'))
  f_tokens = []
  
  for w in tokenst:
    if w not in stop:
      f_tokens.append(w)
  
  return f_tokens

_tokenise and lemmatize_

In [ ]:
from nltk.stem import WordNetLemmatizer

lm = WordNetLemmatizer()

In [ ]:
def tokenise(line):
  i = 0
  lem_words = []

  tokens = line.split()
  f_tokens = removeStop(tokens)
  
  for j in f_tokens:
    lem_words.append(lm.lemmatize(j))
    
  return lem_words

In [ ]:
tokenise("hello i am good")

['hello', 'good']

**Pre-processing the train_data**

In [ ]:
#call all the above defined functions
def preprocess_tweets(data):

  cleaned_tweets = defaultdict()
  vocab = set()

  for index,row in data.iterrows():
    text = row['Tweet_text']
    id =  row['ID']
    #cleaning tweets
    tweet = remove_mentions(text)
    tweet = remove_url(tweet)
    tweet = remove_specialChars(tweet)
    tweet = detect_elongated_words(tweet)
    tweet = expand_contractions(tweet)
    tweet = handle_verbForms(tweet)

    cleaned_tweets[id] = tweet

    #generate tokens
    tokens = tokenise(tweet)
    for i in tokens:
      vocab.add(i)

  return cleaned_tweets,vocab


In [ ]:
cleaned_anger,vocab_anger = preprocess_tweets(anger_train)
cleaned_joy,vocab_joy = preprocess_tweets(joy_train)

######**Feature Extraction**

**_VADER Sentiment_**

In [ ]:
def detect_sentiment(sentence):
  sentence = remove_hash(sentence)

  senti = SentimentIntensityAnalyzer() 
  sentiment_score = senti.polarity_scores(sentence) 
  
  return sentiment_score
  # print("Sentiment scores are: ", sentiment_score) 

In [ ]:
detect_sentiment("My mood is not good")

{'compound': -0.3412, 'neg': 0.376, 'neu': 0.624, 'pos': 0.0}

**_Lexicon based features_**



>**1. Polar word count**



In [ ]:
file1 = "/content/drive/My Drive/NLP_AssgnData/Lexicons_csv/BingLiu.csv"
file2 = "/content/drive/My Drive/NLP_AssgnData/Lexicons_csv/MPQA.csv"

In [ ]:
# p_df1 = pd.read_csv(file1,  names=["Word", "Polarity"],sep='\t')
p_df2 = pd.read_csv(file2)
p_df1 = pd.read_csv(file1)

In [ ]:
p_df1

,Word,Polarity
0,a+,positive
1,abound,positive
2,abounds,positive
3,abundance,positive
4,abundant,positive
...,...,...
6784,zaps,negative
6785,zealot,negative
6786,zealous,negative
6787,zealously,negative


In [ ]:
p_df2

,Word,Polarity
0,abandoned,negative
1,abandonment,negative
2,abandon,negative
3,abase,negative
4,abasement,negative
...,...,...
8216,zealot,negative
8217,zealous,negative
8218,zealously,negative
8219,zenith,positive


In [ ]:
polarity_df = p_df1.append(p_df2)
polarity_df

,Word,Polarity
0,a+,positive
1,abound,positive
2,abounds,positive
3,abundance,positive
4,abundant,positive
...,...,...
8216,zealot,negative
8217,zealous,negative
8218,zealously,negative
8219,zenith,positive


In [ ]:
word_polarity_dict = defaultdict(list)

for index,row in polarity_df.iterrows():
  word = row['Word']
  polarity = row['Polarity']
  
  word_polarity_dict[word].append(polarity)

word_polarity_dict

defaultdict(list,
            {'a+': ['positive'],
             'abound': ['positive', 'positive'],
             'abounds': ['positive'],
             'abundance': ['positive', 'positive'],
             'abundant': ['positive', 'positive'],
             'accessable': ['positive'],
             'accessible': ['positive', 'positive'],
             'acclaim': ['positive', 'positive', 'positive'],
             'acclaimed': ['positive', 'positive'],
             'acclamation': ['positive', 'positive'],
             'accolade': ['positive', 'positive'],
             'accolades': ['positive', 'positive'],
             'accommodative': ['positive', 'positive'],
             'accomodative': ['positive'],
             'accomplish': ['positive', 'positive'],
             'accomplished': ['positive'],
             'accomplishment': ['positive', 'positive'],
             'accomplishments': ['positive', 'positive'],
             'accurate': ['positive', 'positive'],
             'accurately': ['posi

In [ ]:
def polarity_count(sent):
  sent = remove_hash(sent)

  polarityCount = Counter()
  for token in sent.split():
    polarityCount += Counter(word_polarity_dict[token])
  return polarityCount



> **2. Aggregate polarity scores**



_load AFINN lexicon_

In [ ]:
afinn_df = pd.read_csv("/content/drive/My Drive/NLP_AssgnData/Lexicons_csv/AFINN-word.csv")
afinn_df

,Word,Score
0,abandon,-2
1,abandoned,-2
2,abandons,-2
3,abducted,-2
4,abduction,-2
...,...,...
3377,yucky,-2
3378,yummy,3
3379,zealot,-2
3380,zealots,-2


_load Sentiment140 lexicon_

In [ ]:
senti140_df = pd.read_csv("/content/drive/My Drive/NLP_AssgnData/Lexicons_csv/Sentiment140.csv")
senti140_df = senti140_df[['Word','sentimentScore']]
senti140_df.rename(columns = {'sentimentScore':'Score'}, inplace = True)
senti140_df

,Word,Score
0,@jeffrey_donovan,5
1,familar,5
2,@vppatel2011,5
3,emilio,5
4,@livetolovemcfly,5
...,...,...
29942,stephenson,-0.461
29943,#work,-0.461
29944,@therealboyzone,-0.461
29945,ills,-0.461


_load SentiWordNet lexicon_

In [ ]:
sentiWordnet_df = pd.read_csv("/content/drive/My Drive/NLP_AssgnData/Lexicons_csv/SentiWordNet.csv")

sentiWordnet_df = sentiWordnet_df[['PosScore','NegScore','SynsetTerms']]
sentiWordnet_df

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,PosScore,NegScore,SynsetTerms
0,0.125,0.000,able#1
1,0.000,0.750,unable#1
2,0.000,0.000,dorsal#2 abaxial#1
3,0.000,0.000,ventral#2 adaxial#1
4,0.000,0.000,acroscopic#1
...,...,...,...
117649,0.000,0.125,fog_up#1
117650,0.000,0.000,coal#1 char#1
117651,0.125,0.250,haze#1
117652,0.125,0.000,deflagrate#1


_combine all the lexicons_

In [ ]:
pol_score_df = afinn_df.append(senti140_df)
pol_score_df

,Word,Score
0,abandon,-2
1,abandoned,-2
2,abandons,-2
3,abducted,-2
4,abduction,-2
...,...,...
29942,stephenson,-0.461
29943,#work,-0.461
29944,@therealboyzone,-0.461
29945,ills,-0.461


In [ ]:
word_sentiment_score = defaultdict()

for index,row in pol_score_df.iterrows():
  word = row['Word']
  sentScore = row['Score']
  
  word_sentiment_score[word] = sentScore

word_sentiment_score

defaultdict(None,
            {'abandon': -2,
             'abandoned': -2,
             'abandons': -2,
             'abducted': -2,
             'abduction': -2,
             'abductions': -2,
             'abhor': -3,
             'abhorred': -3,
             'abhorrent': -3,
             'abhors': -3,
             'abilities': '0.619',
             'ability': '-0.268',
             'aboard': 1,
             'aborted': '-0.356',
             'aborts': -1,
             'absentee': -1,
             'absentees': -1,
             'absolve': 2,
             'absolved': 2,
             'absolves': 2,
             'absolving': 2,
             'absorbed': 1,
             'abuse': -3,
             'abused': -3,
             'abuses': -3,
             'abusing': -3,
             'abusive': '-0.386',
             'accept': '-0.273',
             'acceptable': 1,
             'acceptance': '0.872',
             'accepted': 1,
             'accepting': 1,
             'accepts': '-0.443',
      

In [ ]:
def agg_polarityScore(sent):
  sent = remove_hash(sent)
  
  sentScore = {}
  pos = 0
  neg = 0

  for token in sent.split():
    try:
      score = word_sentiment_score[token]
      if(score<0):
        neg +=score
      else:
        pos +=score
    except:
      pass

  pos = pos/len(sent.split())
  neg = neg/len(sent.split())

  sentScore['positive'] = pos
  sentScore['negative'] = neg

  return sentScore



> **3. Aggregate polarity scores(Hashtags)**



_load NRC Hashtag Sentiment lexicon_

In [ ]:
NRC_hash_senti = pd.read_csv("/content/drive/My Drive/NLP_AssgnData/Lexicons_csv/NRC_Hashtag_Sentiment.csv")
NRC_hash_senti

,Hashtag,sentimentScore,numPositive,numNegative,Unnamed: 4
0,#fabulous,7.526,2301.0,2,NaN
1,#excellent,7.247,2612.0,3,NaN
2,#superb,7.199,1660.0,2,NaN
3,#perfection,7.099,3004.0,4,NaN
4,#terrific,6.922,629.0,1,NaN
...,...,...,...,...,...
54124,#rotten,-6.211,2.0,1608,NaN
54125,#pathetic,-6.302,7.0,6168,NaN
54126,ipad2,-6.615,1.0,1205,NaN
54127,#dreadful,-6.764,1.0,1398,NaN


In [ ]:
# i have considered only the sentiment score of the hashtag
hash_polarity = defaultdict()

for index,row in NRC_hash_senti.iterrows():
  word = row['Hashtag']
  score = row['sentimentScore']
  hash_polarity[word] = score

In [ ]:
def polarity_hash(sent):
  regex = '#[A-Za-z0-9]+'             
  match = re.findall(regex,sent)         #find all the occurances of #tags
  polarity_score = {}
  pos = 0
  neg = 0

  for item in match:
    try:
      score = hash_polarity[item]
      if(score>0):
        pos+=score
      else:
        neg+=score
    except:
      pass
    
  polarity_score['positive'] = pos
  polarity_score['negative'] = neg
  return polarity_score

In [ ]:
polarity_hash("my anme isd #rotten #dreadful its a #superb situation")

{'negative': -12.975, 'positive': 7.199}



> **4. Emotion word count**



_load NRC Word-Emotion Association Lexicon_

In [ ]:
file3 = "/content/drive/My Drive/NLP_AssgnData/Lexicons_csv/NRC_word_emotion_lexicon.csv"
emolex_df = pd.read_csv(file3)
emolex_df.head(11)

,Word,Emotion,Association
0,smut,anger,0
1,smut,anticipation,0
2,smut,disgust,1
3,smut,fear,1
4,smut,joy,0
5,smut,negative,1
6,smut,positive,0
7,smut,sadness,0
8,smut,surprise,0
9,smut,trust,0


In [ ]:
word_emotion_dict = defaultdict(list)

for index,row in emolex_df.iterrows():
  word = row['Word']
  emotion = row['Emotion']
  
  if(row['Association']==1):
    word_emotion_dict[word].append(emotion)

In [ ]:
word_emotion_dict

defaultdict(list,
            {'smut': ['disgust', 'fear', 'negative'],
             'expletive': ['anger', 'negative'],
             'greeting': ['positive', 'surprise'],
             'measles': ['disgust', 'fear', 'negative', 'sadness'],
             'proven': ['trust'],
             'inept': ['anger', 'disgust', 'negative'],
             'perverted': ['disgust', 'negative'],
             'inconsequential': ['negative', 'sadness'],
             'unfulfilled': ['anger',
              'anticipation',
              'negative',
              'sadness',
              'surprise'],
             'tantalizing': ['anticipation',
              'joy',
              'negative',
              'positive',
              'surprise'],
             'lynch': ['anger', 'disgust', 'fear', 'negative', 'sadness'],
             'constrained': ['negative'],
             'agitation': ['anger', 'negative'],
             'hash': ['negative'],
             'trump': ['surprise'],
             'gray': ['disgust', '

In [ ]:
def generate_emotion_count(sent):
  sent = remove_hash(sent)
  
  emoCount = Counter()
  for token in sent.split():
    emoCount += Counter(word_emotion_dict[token])
  return emoCount

In [ ]:
generate_emotion_count("I am a good girl")

Counter({'anticipation': 1,
         'joy': 1,
         'positive': 1,
         'surprise': 1,
         'trust': 1})



> **5. Aggregate emotion score**



_load NRC-10 Expanded lexicon_

In [ ]:
NRClex_df = pd.read_csv("/content/drive/My Drive/NLP_AssgnData/Lexicons_csv/NRC_10_expanded.csv")
NRClex_df

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,!,0.032954588390801,0.102526,0.039617,0.021636,0.184542,0.076891,0.300205,0.005777,0.042447,0.036114,NaN,NaN,NaN,NaN,NaN,NaN
1,!!,0.050962844050056,0.092440,0.035688,0.017048,0.116958,0.076364,0.201673,0.004525,0.034836,0.030504,NaN,NaN,NaN,NaN,NaN,NaN
2,!!!,0.079344109142187,0.091714,0.040714,0.022238,0.103374,0.090020,0.145371,0.004590,0.028733,0.024017,NaN,NaN,NaN,NaN,NaN,NaN
3,!!!!,0.086259347402643,0.077819,0.047756,0.020853,0.079306,0.099657,0.098460,0.005919,0.019912,0.023743,NaN,NaN,NaN,NaN,NaN,NaN
4,!!!!!,0.07263538041823,0.072729,0.037728,0.022970,0.072132,0.088789,0.088432,0.005933,0.019227,0.024550,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42893,~~~,0.0464341,0.093462,0.019366,0.032560,0.075316,0.235931,0.071702,0.026766,0.031877,0.034241,NaN,NaN,NaN,NaN,NaN,NaN
42894,~~~>,0.0301983,0.073687,0.018303,0.020097,0.119348,0.120536,0.277235,0.008276,0.042328,0.059985,NaN,NaN,NaN,NaN,NaN,NaN
42895,~~~~,0.0283111,0.091188,0.043933,0.024337,0.182899,0.154934,0.166007,0.009597,0.038416,0.019321,NaN,NaN,NaN,NaN,NaN,NaN
42896,~~~~~,0.0331622,0.120175,0.063907,0.017901,0.119867,0.294683,0.088559,0.018311,0.046223,0.010135,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
word_lex_score = defaultdict(lambda:defaultdict())

for index,row in NRClex_df.iterrows():
  word = row['word']

  word_lex_score[word]["anger"] = row['anger']
  word_lex_score[word]["joy"] = row['joy']

In [ ]:
word_lex_score

defaultdict(<function __main__.<lambda>>,
            {'!': defaultdict(None,
                         {'anger': '0.2076181162944',
                          'joy': 0.007170098753317}),
             '!!': defaultdict(None,
                         {'anger': '0.045121065611428',
                          'joy': 0.014251945770231}),
             '!!!': defaultdict(None,
                         {'anger': '0.058601329641997',
                          'joy': 0.006319701236449999}),
             '!!!!': defaultdict(None,
                         {'anger': '0.039287520708372',
                          'joy': 0.010874721527341}),
             '!!!!!': defaultdict(None,
                         {'anger': '0.07263538041823',
                          'joy': 0.072131950059369}),
             '!!!!!!': defaultdict(None,
                         {'anger': '0.097419410874322',
                          'joy': 0.059055092278755994}),
             '!!!!!!!': defaultdict(None,
                      

In [ ]:
def agg_emotionscore_nohash(sent):
  sent = remove_hash(sent)

  emotionScore = {}
  anger = 0
  joy = 0

  for token in sent.split():
    try:
      anger += float(word_lex_score[token]['anger'])
      joy += float(word_lex_score[token]['joy'])
    except:
      pass
  
  agg_anger = anger/len(sent.split())
  agg_joy = joy/len(sent.split())

  emotionScore['anger']=agg_anger
  emotionScore['joy']=agg_joy

  return emotionScore

In [ ]:
agg_emotionscore_nohash("my team is not good")

{'anger': 0.05533976862318181, 'joy': 0.007095525260249801}



> **6. Aggregate emotion score(Hashtags)**

_load NRC
Hashtag Emotion Association Lexicon_

In [ ]:
NRC_hashtag_df = pd.read_csv("/content/drive/My Drive/NLP_AssgnData/Lexicons_csv/NRC_Hashtag_EmotionAssociation_Lexicon.csv")
NRC_hashtag_df

,Emotion,Hashtag,AssociationScore
0,anticipation,crae,2.237478
1,anticipation,#mycolour,2.237478
2,anticipation,#vigilance,2.237478
3,anticipation,#vigilant,2.237478
4,anticipation,#hmmmmm,2.237478
...,...,...,...
32384,disgust,sent,0.000658
32385,disgust,laughing,0.000328
32386,disgust,status,0.000108
32387,disgust,ya,0.000018


In [ ]:
anger_df = NRC_hashtag_df.loc[NRC_hashtag_df['Emotion']=='anger']
joy_df = NRC_hashtag_df.loc[NRC_hashtag_df['Emotion']=='joy']

hashtag_association = defaultdict(lambda:defaultdict())

for index,row in anger_df.iterrows():
  word = row['Hashtag']
  hashtag_association[word]["anger"] = row['AssociationScore']

for index,row in joy_df.iterrows():
  word = row['Hashtag']
  hashtag_association[word]["joy"] = row['AssociationScore']

hashtag_association

defaultdict(<function __main__.<lambda>>,
            {'#agitated': defaultdict(None, {'anger': 1.42991541495012}),
             '#roar': defaultdict(None, {'anger': 1.42991541495012}),
             'revving': defaultdict(None, {'anger': 1.42991541495012}),
             '#backgroundcheck': defaultdict(None,
                         {'anger': 1.42991541495012}),
             '#hormonal': defaultdict(None, {'anger': 1.42991541495012}),
             '#hangry': defaultdict(None, {'anger': 1.42991541495012}),
             'darkside': defaultdict(None, {'anger': 1.42991541495012}),
             '#imangry': defaultdict(None, {'anger': 1.42991541495012}),
             '#point': defaultdict(None, {'anger': 1.42991541495012}),
             'times&lt': defaultdict(None, {'anger': 1.42991541495012}),
             '#pistoff': defaultdict(None, {'anger': 1.42991541495012}),
             'dents': defaultdict(None, {'anger': 1.42991541495012}),
             'mts': defaultdict(None, {'anger': 1.4299154

In [ ]:
def agg_emotionscore_hash(sent):

  emotion_hashScore = {}
  anger = 0
  joy = 0

  for token in sent.split():
    try:
      anger += float(hashtag_association[token]['anger'])
      joy += float(hashtag_association[token]['joy'])
    except:
      pass
    
  agg_anger = anger/len(sent.split())
  agg_joy = joy/len(sent.split())

  emotion_hashScore['anger']=agg_anger
  emotion_hashScore['joy']=agg_joy

  return emotion_hashScore



> **7. Emoticons Score**



_create emoji to emoticon dictionary_

In [ ]:
#reference: https://gist.github.com/lorenzOliveto/f20a89e9f68276cae21497a177ad8a4c

to_smiley = {
    "o/": "👋",
    "</3": "💔",
    "<3": "💗",
    "8-D": "😁",
    "8D": "😁",
    ":-D": "😁",
    ":-3": "😁",
    ":3": "😁",
    ":D": "😁",
    "B^D": "😁",
    "X-D": "😁",
    "XD": "😁",
    "x-D": "😁",
    "xD": "😁",
    ":\")": "😂",
    ":\"-)": "😂",
    ":-))": "😃",
    "8)": "😄",
    ":)": "😊",
    ":-)": "😄",
    ":]": "😄",
    ":^)": "😄",
    ":c)": "😄",
    ":o)": "😄",
    ":}": "😄",
    ":っ)": "😄",
    "0:)": "😇",
    "0:-)": "😇",
    "0:-3": "😇",
    "0:3": "😇",
    "0;^)": "😇",
    "O:-)": "😇",
    "3:)": "😈",
    "3:-)": "😈",
    "}:)": "😈",
    "}:-)": "😈",
    "*)": "😉",
    "*-)": "😉",
    ":-,": "😉",
    ";)": "😉",
    ";-)": "😉",
    ";-]": "😉",
    ";D": "😉",
    ";]": "😉",
    ";^)": "😉",
    ":-|": "😐",
    ":|": "😐",
    ":(": "😒",
    ":-(": "😒",
    ":-<": "😒",
    ":-[": "😒",
    ":-c": "😒",
    ":<": "😒",
    ":[": "😒",
    ":c": "😒",
    ":{": "😒",
    ":っC": "😒",
    "%)": "😖",
    "%-)": "😖",
    ":-P": "😜",
    ":-b": "😜",
    ":-p": "😜",
    ":-Þ": "😜",
    ":-þ": "😜",
    ":P": "😜",
    ":b": "😜",
    ":p": "😜",
    ":Þ": "😜",
    ":þ": "😜",
    ";(": "😜",
    "X-P": "😜",
    "XP": "😜",
    "d:": "😜",
    "x-p": "😜",
    "xp": "😜",
    ":-||": "😠",
    ":@": "😠",
    ":-.": "😡",
    ":-/": "😡",
    ":/": "😡",
    ":L": "😡",
    ":S": "😡",
    ":\\": "😡",
    ":\"(": "😢",
    ":\"-(": "😢",
    "^5": "😤",
    "^<_<": "😤",
    "o/\\o": "😤",
    "|-O": "😫",
    "|;-)": "😫",
    ":###..": "😰",
    ":-###..": "😰",
    "D-\":": "😱",
    "D8": "😱",
    "D:": "😱",
    "D:<": "😱",
    "D;": "😱",
    "DX": "😱",
    "v.v": "😱",
    "8-0": "😲",
    ":-O": "😲",
    ":-o": "😲",
    ":O": "😲",
    ":o": "😲",
    "O-O": "😲",
    "O_O": "😲",
    "O_o": "😲",
    "o-o": "😲",
    "o_O": "😲",
    "o_o": "😲",
    ":$": "😳",
    "#-)": "😵",
    ":#": "😶",
    ":&": "😶",
    ":-#": "😶",
    ":-&": "😶",
    ":-X": "😶",
    ":X": "😶",
    ":-J": "😼",
    ":*": "😽",
    ":^*": "😽",
    "ಠ_ಠ": "🙅",
    "*\\0/*": "🙆",
    "\\o/": "🙆",
    ":>": "😄",
    ">.<": "😡",
    ">:(": "😠",
    ">:)": "😈",
    ">:-)": "😈",
    ">:/": "😡",
    ">:O": "😲",
    ">:P": "😜",
    ">:[": "😒",
    ">:\\": "😡",
    ">;)": "😈",
    ">_>^": "😤",
    "^^": "😊",
    ":sweat": "😅"
}

emojiToemoticon = defaultdict(list)
for i in to_smiley.items():
  emojiToemoticon[i[1]].append(i[0])

emojiToemoticon

defaultdict(list,
            {'👋': ['o/'],
             '💔': ['</3'],
             '💗': ['<3'],
             '😁': ['8-D',
              '8D',
              ':-D',
              ':-3',
              ':3',
              ':D',
              'B^D',
              'X-D',
              'XD',
              'x-D',
              'xD'],
             '😂': [':")', ':"-)'],
             '😃': [':-))'],
             '😄': ['8)', ':-)', ':]', ':^)', ':c)', ':o)', ':}', ':っ)', ':>'],
             '😅': [':sweat'],
             '😇': ['0:)', '0:-)', '0:-3', '0:3', '0;^)', 'O:-)'],
             '😈': ['3:)', '3:-)', '}:)', '}:-)', '>:)', '>:-)', '>;)'],
             '😉': ['*)', '*-)', ':-,', ';)', ';-)', ';-]', ';D', ';]', ';^)'],
             '😊': [':)', '^^'],
             '😐': [':-|', ':|'],
             '😒': [':(',
              ':-(',
              ':-<',
              ':-[',
              ':-c',
              ':<',
              ':[',
              ':c',
              ':{',
              ':っC',
       

In [ ]:
emojiToemoticon['😈']

['3:)', '3:-)', '}:)', '}:-)', '>:)', '>:-)', '>;)']

_load AFINN Emoticon lexicon_

In [ ]:
afinn_emo_df = pd.read_csv("/content/drive/My Drive/NLP_AssgnData/Lexicons_csv/AFINN-emoticon-8.csv")
afinn_emo_df

,Emoticon,Score
0,:),2
1,:(,-2
2,:|,-1
3,:],2
4,:[,-2
...,...,...
91,xD,3
92,xo,3
93,xoxo,3
94,xoxoxo,4


In [ ]:
afinn_emo_dict = defaultdict()
for index,row in afinn_emo_df.iterrows():
  afinn_emo_dict[row['Emoticon']] = row['Score']

afinn_emo_dict

defaultdict(None,
            {'#NAME?': 3,
             '(-:': 2,
             '(:': 2,
             ')-:': -2,
             '):': -2,
             '8(': -2,
             '8)': 2,
             '8-(': -2,
             '8-)': 2,
             '8-D': 3,
             '8D': 3,
             ":'(": -2,
             ':(': -2,
             ':((': -3,
             ':)': 2,
             ':))': 3,
             ':)))': 3,
             ':))))': 3,
             ':)))))': 3,
             ':))))))': 4,
             ':)))))))': 4,
             ':))))))))': 4,
             ':)))))))))': 4,
             ':))))))))))': 4,
             ':*': 2,
             ':-(': -2,
             ':-((': -3,
             ':-(((': -3,
             ':-((((': -3,
             ':-)': 2,
             ':-))': 3,
             ':-)))': 3,
             ':-))))': 3,
             ':-)))))': 3,
             ':-))))))': 4,
             ':-)))))))': 4,
             ':-))))))))': 4,
             ':-)))))))))': 4,
             ':-*': 2,
 

In [ ]:
# https://pypi.org/project/afinn/

def emoticons_score(sent):
  sent = remove_hash(sent)
  score_dict = {}
  pos = 0
  neg = 0

  for token in sent.split():
    try:
      emoticons = emojiToemoticon[token]
      # print(emoticons)
      for i in emoticons:
        # print(i)
        score = afinn_emo_dict[i]
        if(int(score) > 0):
          pos +=score
        else:
          neg +=score
    except:
      pass

  score_dict['positive'] = pos
  score_dict['negative'] = pos

  return score_dict

In [ ]:
emoticons_score("I am good 😒")


{'negative': 0, 'positive': 0}

In [ ]:
("I am good 👿").split()

['I', 'am', 'good', '👿']



> **8. Negation count**



In [ ]:
def count_negatives(sent):
  negative_words = ['neither','nobody','never','no','nothing','nowhere','none','not','haven\'t','hasn\'t','can\'t','couldn\'t','shouldn\'t','won\'t','wouldn\'t','don\'t','doesn\'t','didn\'t','isn\'t','aren\'t','ain\'t']
  count = 0

  for word in sent.split():
    if word in negative_words:
      count +=1

  neg_dict = {"negative_counts":count}
  return neg_dict

In [ ]:
count_negatives("I haven't gone there. She never came.")

{'negative_counts': 2}

######**Create feature matrix - train data**

In [ ]:
#applying all the above functions on cleaned tweets
# after receiving dataframe for each feature just concatenate all
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html

def extract_features(cleaned_tweets):
  #VADER feature
  vader = [detect_sentiment(tweet[1]) for tweet in cleaned_tweets.items()]
  vader = pd.DataFrame(vader, index = (tweet[0] for tweet in cleaned_tweets.items()))

  #feature 1
  pol_counts = [polarity_count(tweet[1]) for tweet in cleaned_tweets.items()]
  pol_counts = pd.DataFrame(pol_counts, index = (tweet[0] for tweet in cleaned_tweets.items()))

  #feature 2
  agg_polarity = [agg_polarityScore(tweet[1]) for tweet in cleaned_tweets.items()]
  agg_polarity = pd.DataFrame(agg_polarity, index = (tweet[0] for tweet in cleaned_tweets.items()))

  #feature 3
  agg_polarity_hash = [polarity_hash(tweet[1]) for tweet in cleaned_tweets.items()]
  agg_polarity_hash = pd.DataFrame(agg_polarity_hash, index = (tweet[0] for tweet in cleaned_tweets.items()))

  #feature 4
  emotion_word_counts = [generate_emotion_count(tweet[1]) for tweet in cleaned_tweets.items()]
  emotion_count = pd.DataFrame(emotion_word_counts, index = (tweet[0] for tweet in cleaned_tweets.items()))
  emotion_count = emotion_count.fillna(0)

  #feature 5
  agg_score = [agg_emotionscore_nohash(tweet[1]) for tweet in cleaned_tweets.items()]
  agg_score = pd.DataFrame(agg_score, index = (tweet[0] for tweet in cleaned_tweets.items()))
  agg_score = agg_score.fillna(0)

  #feature 6
  agg_score_hash = [agg_emotionscore_hash(tweet[1]) for tweet in cleaned_tweets.items()]
  agg_score_hash = pd.DataFrame(agg_score_hash, index = (tweet[0] for tweet in cleaned_tweets.items()))

  #feature 7
  afinn_scores = [emoticons_score(tweet[1]) for tweet in cleaned_tweets.items()]
  afinn_scores = pd.DataFrame(afinn_scores, index = (tweet[0] for tweet in cleaned_tweets.items()))

  #feature 8
  neg_counts = [count_negatives(tweet[1]) for tweet in cleaned_tweets.items()]
  neg_counts = pd.DataFrame(neg_counts, index = (tweet[0] for tweet in cleaned_tweets.items()))
 
  #combine all the features
  features = pd.concat([vader,pol_counts,agg_polarity,agg_polarity_hash,emotion_count,agg_score,agg_score_hash,afinn_scores,neg_counts],axis=1)

  return features

In [ ]:
features_anger = extract_features(cleaned_anger)
features_joy = extract_features(cleaned_joy)

In [ ]:
features_anger = features_anger.fillna(0)
features_anger

,neg,neu,pos,compound,negative,neutral,positive,positive,negative,positive,negative,anger,disgust,negative,fear,sadness,positive,trust,anticipation,surprise,joy,anger,joy,anger,joy,positive,negative,negative_counts
10000,0.302,0.698,0.000,-0.7579,10.0,1.0,0.0,0.0,-0.352941,0,0,2.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.071142,0.011507,0.274581,0.000000,0,0,0
10001,0.112,0.888,0.000,-0.5267,2.0,2.0,3.0,0.0,-0.107143,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.066149,0.018505,0.053880,0.000000,0,0,1
10002,0.000,1.000,0.000,0.0000,0.0,0.0,0.0,0.0,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.039556,0.020747,0.099486,0.000000,0,0,1
10003,0.613,0.387,0.000,-0.9712,15.0,1.0,1.0,0.0,-0.826087,0,0,2.0,1.0,6.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.071709,0.011509,0.240726,0.000000,0,0,0
10004,0.115,0.885,0.000,-0.4588,3.0,0.0,0.0,0.0,-0.083333,0,0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,0.069356,0.024750,0.085291,0.000000,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10852,0.000,0.682,0.318,0.6369,2.0,0.0,2.0,0.1,0.000000,0,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037578,0.060249,0.000000,0.000000,0,0,0
10853,0.000,1.000,0.000,0.0000,0.0,0.0,0.0,0.0,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.051871,0.007913,0.068856,0.119927,0,0,0
10854,0.000,0.575,0.425,0.5719,0.0,0.0,3.0,0.0,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.020743,0.019209,0.033002,0.000000,0,0,0
10855,0.159,0.841,0.000,-0.1779,4.0,1.0,0.0,0.0,-0.100000,0,0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.020610,0.014943,0.000000,0.000000,0,0,0


In [ ]:
features_joy = features_joy.fillna(0)
features_joy

,neg,neu,pos,compound,positive,neutral,negative,positive,negative,positive,negative,joy,positive,surprise,anticipation,trust,negative,sadness,anger,fear,disgust,anger,joy,anger,joy,positive,negative,negative_counts
30000,0.154,0.697,0.148,-0.0314,6.0,3.0,1.0,0.000000,-0.125000,0,0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040265,0.052237,0.035744,0.0,0,0,0
30001,0.059,0.650,0.291,0.8176,2.0,4.0,1.0,0.142857,0.000000,0,0,1.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.036388,0.025009,0.038688,0.0,0,0,1
30002,0.000,0.829,0.171,0.5106,3.0,0.0,1.0,0.000000,0.000000,0,0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.026500,0.034265,0.000000,0.0,0,0,0
30003,0.000,0.393,0.607,0.9866,17.0,3.0,1.0,0.000000,0.000000,0,0,4.0,5.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.024781,0.086617,0.015975,0.0,0,0,0
30004,0.000,0.535,0.465,0.9552,9.0,2.0,0.0,0.000000,0.000000,0,0,4.0,4.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.031185,0.039008,0.019703,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30818,0.216,0.784,0.000,-0.5267,3.0,0.0,4.0,0.000000,-0.222222,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.048357,0.019790,0.025002,0.0,0,0,0
30819,0.252,0.407,0.341,0.2732,2.0,0.0,3.0,0.000000,-0.285714,0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.084697,0.013153,0.033039,0.0,0,0,0
30820,0.141,0.573,0.286,0.4215,4.0,0.0,2.0,0.105263,-0.157895,0,0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.057567,0.019992,0.044592,0.0,0,0,0
30821,0.135,0.700,0.165,0.1531,3.0,1.0,0.0,0.000000,0.000000,0,0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.051848,0.018547,0.132592,0.0,0,0,0


In [ ]:
feature_matrix_anger = features_anger.to_numpy()
print(feature_matrix_anger.shape)
feature_matrix_anger

(857, 28)


array([[0.302, 0.698, 0.   , ..., 0.   , 0.   , 0.   ],
       [0.112, 0.888, 0.   , ..., 0.   , 0.   , 1.   ],
       [0.   , 1.   , 0.   , ..., 0.   , 0.   , 1.   ],
       ...,
       [0.   , 0.575, 0.425, ..., 0.   , 0.   , 0.   ],
       [0.159, 0.841, 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.597, 0.403, ..., 0.   , 0.   , 0.   ]])

In [ ]:
feature_matrix_joy = features_joy.to_numpy()
print(feature_matrix_joy.shape)
feature_matrix_joy

(823, 28)


array([[0.154, 0.697, 0.148, ..., 0.   , 0.   , 0.   ],
       [0.059, 0.65 , 0.291, ..., 0.   , 0.   , 1.   ],
       [0.   , 0.829, 0.171, ..., 0.   , 0.   , 0.   ],
       ...,
       [0.141, 0.573, 0.286, ..., 0.   , 0.   , 0.   ],
       [0.135, 0.7  , 0.165, ..., 0.   , 0.   , 0.   ],
       [0.328, 0.672, 0.   , ..., 0.   , 0.   , 0.   ]])

######**Create feature matrix - test data**

**pre-processing**

In [ ]:
clean_test_anger,vocab_anger= preprocess_tweets(anger_test)
clean_test_joy,vocab_joy= preprocess_tweets(joy_test)

**extract features**

In [ ]:
features_test_anger = extract_features(clean_test_anger)
features_test_anger = features_test_anger.fillna(0)
features_test_anger

,neg,neu,pos,compound,positive,negative,neutral,both,positive,negative,positive,negative,anticipation,joy,positive,trust,anger,negative,sadness,disgust,surprise,fear,anger,joy,anger,joy,positive,negative,negative_counts
10941,0.000,1.000,0.000,0.0000,2.0,0.0,0.0,0.0,0.000000,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.051153,0.016543,0.073605,0.022843,0,0,0
10942,0.000,0.893,0.107,0.0516,0.0,3.0,0.0,0.0,0.000000,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.038577,0.017155,0.004415,0.000000,0,0,0
10943,0.160,0.840,0.000,-0.6369,0.0,3.0,1.0,0.0,0.000000,-0.173913,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.063276,0.016641,0.116516,0.000000,0,0,0
10944,0.000,0.805,0.195,0.7003,4.0,2.0,0.0,0.0,0.086957,0.000000,0,0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052595,0.032315,0.059113,0.000000,0,0,1
10945,0.000,1.000,0.000,0.0000,0.0,0.0,1.0,0.0,0.000000,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041263,0.044057,0.071825,0.000000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,0.401,0.599,0.000,-0.7717,0.0,4.0,0.0,0.0,0.000000,-0.166667,0,0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.152693,0.016683,0.098358,0.009319,0,0,0
11697,0.250,0.750,0.000,-0.2500,2.0,0.0,0.0,0.0,0.000000,-0.285714,0,0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.036727,0.012012,0.021283,0.000000,0,0,0
11698,0.000,0.852,0.148,0.3818,3.0,2.0,0.0,0.0,0.000000,0.000000,0,0,0.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.028091,0.012234,0.093163,0.000000,0,0,0
11699,0.000,0.840,0.160,0.4215,2.0,2.0,2.0,0.0,0.045455,0.000000,0,0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.032954,0.015629,0.088783,0.000000,0,0,2


In [ ]:
features_test_anger = features_test_anger.drop(columns=['both'])
features_test_anger

,neg,neu,pos,compound,positive,negative,neutral,positive,negative,positive,negative,anticipation,joy,positive,trust,anger,negative,sadness,disgust,surprise,fear,anger,joy,anger,joy,positive,negative,negative_counts
10941,0.000,1.000,0.000,0.0000,2.0,0.0,0.0,0.000000,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.051153,0.016543,0.073605,0.022843,0,0,0
10942,0.000,0.893,0.107,0.0516,0.0,3.0,0.0,0.000000,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.038577,0.017155,0.004415,0.000000,0,0,0
10943,0.160,0.840,0.000,-0.6369,0.0,3.0,1.0,0.000000,-0.173913,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.063276,0.016641,0.116516,0.000000,0,0,0
10944,0.000,0.805,0.195,0.7003,4.0,2.0,0.0,0.086957,0.000000,0,0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052595,0.032315,0.059113,0.000000,0,0,1
10945,0.000,1.000,0.000,0.0000,0.0,0.0,1.0,0.000000,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041263,0.044057,0.071825,0.000000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,0.401,0.599,0.000,-0.7717,0.0,4.0,0.0,0.000000,-0.166667,0,0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.152693,0.016683,0.098358,0.009319,0,0,0
11697,0.250,0.750,0.000,-0.2500,2.0,0.0,0.0,0.000000,-0.285714,0,0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.036727,0.012012,0.021283,0.000000,0,0,0
11698,0.000,0.852,0.148,0.3818,3.0,2.0,0.0,0.000000,0.000000,0,0,0.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.028091,0.012234,0.093163,0.000000,0,0,0
11699,0.000,0.840,0.160,0.4215,2.0,2.0,2.0,0.045455,0.000000,0,0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.032954,0.015629,0.088783,0.000000,0,0,2


In [ ]:
features_test_anger = features_test_anger.to_numpy()
print(features_test_anger.shape)
features_test_anger

(760, 28)


array([[0.   , 1.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.893, 0.107, ..., 0.   , 0.   , 0.   ],
       [0.16 , 0.84 , 0.   , ..., 0.   , 0.   , 0.   ],
       ...,
       [0.   , 0.852, 0.148, ..., 0.   , 0.   , 0.   ],
       [0.   , 0.84 , 0.16 , ..., 0.   , 0.   , 2.   ],
       [0.239, 0.695, 0.066, ..., 0.   , 0.   , 0.   ]])

In [ ]:
features_test_joy = extract_features(clean_test_joy)
features_test_joy = features_test_joy.fillna(0)
features_test_joy

,neg,neu,pos,compound,neutral,positive,negative,both,positive,negative,positive,negative,anticipation,joy,positive,trust,surprise,anger,disgust,fear,negative,sadness,anger,joy,anger,joy,positive,negative,negative_counts
30902,0.000,0.390,0.610,0.8555,1.0,6.0,0.0,0.0,0.222222,0.000000,0,0,1.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.013401,0.027365,0.008210,0.000000,0,0,0
30903,0.000,0.671,0.329,0.5994,0.0,3.0,0.0,0.0,0.000000,0.000000,0,0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.016420,0.020883,0.000000,0.000000,0,0,0
30904,0.180,0.497,0.323,0.3182,2.0,2.0,2.0,0.0,0.000000,-0.375000,0,0,0.0,2.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.008569,0.013604,0.099565,0.000000,0,0,0
30905,0.000,1.000,0.000,0.0000,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017753,0.054183,0.022198,0.000000,0,0,0
30906,0.141,0.582,0.276,0.5935,0.0,3.0,7.0,0.0,0.000000,-0.333333,0,0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.069447,0.018182,0.088281,0.008597,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31611,0.091,0.657,0.251,0.6815,1.0,6.0,4.0,0.0,0.000000,-0.074074,0,0,3.0,3.0,3.0,2.0,3.0,0.0,0.0,0.0,1.0,0.0,0.046016,0.036649,0.033000,0.000000,0,0,0
31612,0.239,0.761,0.000,-0.2960,0.0,0.0,2.0,0.0,0.000000,0.000000,0,0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.016016,0.017008,0.131147,0.000000,0,0,0
31613,0.215,0.667,0.119,-0.3804,4.0,0.0,7.0,0.0,0.000000,-0.190476,0,0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.060086,0.051018,0.032605,0.000000,0,0,0
31614,0.000,0.896,0.104,0.4404,0.0,2.0,1.0,0.0,0.076923,0.000000,0,0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.021699,0.013274,0.012714,0.024148,0,0,0


In [ ]:
features_test_joy = features_test_joy.drop(columns=['both'])
features_test_joy

,neg,neu,pos,compound,neutral,positive,negative,positive,negative,positive,negative,anticipation,joy,positive,trust,surprise,anger,disgust,fear,negative,sadness,anger,joy,anger,joy,positive,negative,negative_counts
30902,0.000,0.390,0.610,0.8555,1.0,6.0,0.0,0.222222,0.000000,0,0,1.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.013401,0.027365,0.008210,0.000000,0,0,0
30903,0.000,0.671,0.329,0.5994,0.0,3.0,0.0,0.000000,0.000000,0,0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.016420,0.020883,0.000000,0.000000,0,0,0
30904,0.180,0.497,0.323,0.3182,2.0,2.0,2.0,0.000000,-0.375000,0,0,0.0,2.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.008569,0.013604,0.099565,0.000000,0,0,0
30905,0.000,1.000,0.000,0.0000,0.0,0.0,0.0,0.000000,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017753,0.054183,0.022198,0.000000,0,0,0
30906,0.141,0.582,0.276,0.5935,0.0,3.0,7.0,0.000000,-0.333333,0,0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.069447,0.018182,0.088281,0.008597,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31611,0.091,0.657,0.251,0.6815,1.0,6.0,4.0,0.000000,-0.074074,0,0,3.0,3.0,3.0,2.0,3.0,0.0,0.0,0.0,1.0,0.0,0.046016,0.036649,0.033000,0.000000,0,0,0
31612,0.239,0.761,0.000,-0.2960,0.0,0.0,2.0,0.000000,0.000000,0,0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.016016,0.017008,0.131147,0.000000,0,0,0
31613,0.215,0.667,0.119,-0.3804,4.0,0.0,7.0,0.000000,-0.190476,0,0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.060086,0.051018,0.032605,0.000000,0,0,0
31614,0.000,0.896,0.104,0.4404,0.0,2.0,1.0,0.076923,0.000000,0,0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.021699,0.013274,0.012714,0.024148,0,0,0


In [ ]:
features_test_joy = features_test_joy.to_numpy()
print(features_test_joy.shape)
features_test_joy

(714, 28)


array([[0.   , 0.39 , 0.61 , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.671, 0.329, ..., 0.   , 0.   , 0.   ],
       [0.18 , 0.497, 0.323, ..., 0.   , 0.   , 0.   ],
       ...,
       [0.215, 0.667, 0.119, ..., 0.   , 0.   , 0.   ],
       [0.   , 0.896, 0.104, ..., 0.   , 0.   , 0.   ],
       [0.   , 0.688, 0.312, ..., 0.   , 0.   , 0.   ]])

######**n-gram feature extraction**

**Anger Train**

In [ ]:
train_anger = [tweet[1] for tweet in cleaned_anger.items()]
test_anger = [tweet[1] for tweet in clean_test_anger.items()]

In [ ]:
vectorizer = CountVectorizer(stop_words=set(stopwords.words('english')),ngram_range=(1,2),binary=True,max_features=100)
nGram_trainAnger = vectorizer.fit_transform(train_anger)
nGram_testAnger = vectorizer.fit_transform(test_anger)

In [ ]:
nGram_trainAnger = nGram_trainAnger.toarray()
nGram_trainAnger.shape

(857, 100)

In [ ]:
feature_matrix_anger = np.concatenate((feature_matrix_anger,nGram_trainAnger),axis = 1)
feature_matrix_anger.shape

(857, 128)

**Anger Test data**

In [ ]:
nGram_testAnger = nGram_testAnger.toarray()
nGram_testAnger.shape

(760, 100)

In [ ]:
features_test_anger = np.concatenate((features_test_anger,nGram_testAnger),axis = 1)
features_test_anger.shape

(760, 128)

**Joy Train data**

In [ ]:
train_joy = [tweet[1] for tweet in cleaned_joy.items()]
test_joy = [tweet[1] for tweet in clean_test_joy.items()]

In [ ]:
vectorizer = CountVectorizer(stop_words=set(stopwords.words('english')),ngram_range=(1,2),binary=True,max_features=100)
nGram_trainJoy = vectorizer.fit_transform(train_joy)
nGram_testJoy = vectorizer.fit_transform(test_joy)

In [ ]:
nGram_trainJoy = nGram_trainJoy.toarray()
nGram_trainJoy.shape

(823, 100)

In [ ]:
feature_matrix_joy = np.concatenate((feature_matrix_joy,nGram_trainJoy),axis = 1)
feature_matrix_joy.shape

(823, 128)

**Joy Test Data**

In [ ]:
nGram_testJoy = nGram_testJoy.toarray()
nGram_testJoy.shape

(714, 100)

**Joy test data**

In [ ]:
features_test_joy = np.concatenate((features_test_joy,nGram_testJoy),axis = 1)
features_test_joy.shape

(714, 128)

######**Training Machine Learning models**

In [ ]:
anger_vals = []
joy_vals = []

for index,row in anger_train.iterrows():
  anger_vals.append(row['Intensity'])

for index,row in joy_train.iterrows():
  joy_vals.append(row['Intensity'])
  
print(len(anger_vals))
print(len(joy_vals))

857
823




> **1. SVR**



In [ ]:
svr_model = SVR()

# For Anger
svr_anger = svr_model.fit(feature_matrix_anger,anger_vals)
# For Joy
svr_joy = svr_model.fit(feature_matrix_joy,joy_vals)



> **2. Decision Tree Regressor**



In [ ]:
tree = DecisionTreeRegressor(criterion='mse',max_depth=3)

# For Anger
tree_anger = tree.fit(feature_matrix_anger,anger_vals)
# For Joy
tree_joy = tree.fit(feature_matrix_joy,joy_vals)



> **3. MLP**



In [ ]:
mlp = MLPRegressor(random_state=1, max_iter=500)

# For Anger
mlp_anger = mlp.fit(feature_matrix_anger,anger_vals)
# For Joy
mlp_joy = mlp.fit(feature_matrix_joy,joy_vals)

######**Predictions of ML models**

In [ ]:
anger_vals_test = []
joy_vals_test = []

for index,row in anger_test.iterrows():
  anger_vals_test.append(row['Intensity'])

for index,row in joy_test.iterrows():
  joy_vals_test.append(row['Intensity'])
  
print(len(anger_vals_test))
print(len(joy_vals_test))

760
714


**Evaluate**

In [ ]:
def evaluate(pred,true):
  pearson, _ = pearsonr(pred, true)
  spearman, _ = spearmanr(pred,true)

  print('Pearsons correlation: %.3f' % pearson)
  print('Spearmans correlation: %.3f' % spearman)

>**1. SVR**

In [ ]:
svr_ang_pred = svr_anger.predict(features_test_anger)
svr_ang_pred

In [ ]:
evaluate(svr_ang_pred,anger_vals_test)

Pearsons correlation: -0.300
Spearmans correlation: -0.296


In [ ]:
svr_joy_pred = svr_joy.predict(features_test_joy)
svr_joy_pred

ValueError: ignored

>**2. DT**

In [ ]:
DT_ang_pred = tree_anger.predict(features_test_anger)
DT_ang_pred

In [ ]:
evaluate(DT_ang_pred,anger_vals_test)

Pearsons correlation: -0.367
Spearmans correlation: -0.397


In [ ]:
dt_joy_pred = tree_joy.predict(features_test_joy)
dt_joy_pred

>**3. MLP**

In [ ]:
MLP_ang_pred = mlp_anger.predict(features_test_anger)
MLP_ang_pred

In [ ]:
MLP_joy_pred = mlp_joy.predict(features_test_joy)
MLP_joy_pred

In [ ]:
evaluate(MLP_ang_pred,anger_vals_test)

Pearsons correlation: -0.064
Spearmans correlation: -0.086
